# Agent沙箱 自定义配置文档

## 描述

Agent沙箱支持通过 `x-custom-config` 扩展配置，允许用户使用自定义镜像、环境变量、资源限制和健康检查等高级功能来创建沙箱。

### 主要特性

- **自定义镜像**: 支持使用企业版或个人版镜像仓库中的自定义镜像
- **环境变量配置**: 支持在沙箱启动时注入自定义环境变量
- **资源限制**: 支持配置 CPU 和内存资源限制
- **端口配置**: 支持配置多个容器端口
- **健康检查**: 支持配置 HTTP 健康检查探针
- **启动命令**: 支持自定义容器启动命令和参数

### 使用场景

- 需要使用特定运行环境的 AI Agent
- 需要预装特定依赖的代码执行环境
- 需要精确控制资源分配的应用
- 需要自定义健康检查的服务

## 安装依赖

In [ ]:
%pip install e2b_code_interpreter

## x-custom-config 数据结构说明

在创建沙箱时，可以通过 `metadata` 字段中的 `x-custom-config` 键传递自定义配置。

### 完整数据结构

```json
{
  "image": "ccr.ccs.tencentyun.com/my-repo/my-image:v1.0",
  "imageRegistryType": "enterprise",
  "imageDigest": "sha256:abc123...",
  "command": ["/bin/sh", "-c"],
  "args": ["./start.sh"],
  "env": [
    {"name": "NODE_ENV", "value": "production"},
    {"name": "DEBUG", "value": "true"}
  ],
  "ports": [
    {"name": "http", "port": 8080, "protocol": "TCP"},
    {"name": "metrics", "port": 9090, "protocol": "TCP"}
  ],
  "resources": {
    "cpu": "2",
    "memory": "4Gi"
  },
  "probe": {
    "httpGet": {
      "path": "/health",
      "port": 8080,
      "scheme": "HTTP"
    },
    "readyTimeoutMs": 30000,
    "probeTimeoutMs": 5000,
    "probePeriodMs": 10000,
    "successThreshold": 1,
    "failureThreshold": 3
  }
}
```

### 字段说明

| 字段 | 类型 | 必填 | 说明 |
|------|------|------|------|
| `image` | string | 否 | 自定义镜像地址 |
| `imageRegistryType` | string | 否 | 镜像仓库类型：`enterprise`（企业版）、`personal`（个人版） |
| `imageDigest` | string | 否 | 镜像摘要（sha256 digest），仅响应时返回 |
| `command` | []string | 否 | 容器启动命令（覆盖镜像 ENTRYPOINT） |
| `args` | []string | 否 | 容器启动参数（覆盖镜像 CMD） |
| `env` | []EnvVar | 否 | 环境变量列表 |
| `ports` | []PortConfig | 否 | 端口配置列表 |
| `resources` | ResourceConfig | 否 | 资源配置（CPU/内存） |
| `probe` | ProbeConfig | 否 | 健康检查配置 |

## 环境配置

In [ ]:
import os
import json
from e2b_code_interpreter import Sandbox

# 在Agent Sandbox控制台中获取API Key以及可用域名
# 可通过环境变量预先设置，或在此处直接修改
if not os.getenv("E2B_DOMAIN"):
    os.environ["E2B_DOMAIN"] = "ap-guangzhou.tencentags.com"
if not os.getenv("E2B_API_KEY"):
    os.environ["E2B_API_KEY"] = "your-api-key"

## 使用自定义镜像创建沙箱

通过 `x-custom-config` 可以指定自定义镜像地址和镜像仓库类型。

In [ ]:
# 定义自定义镜像配置
custom_image_config = {
    "image": "ccr.ccs.tencentyun.com/my-repo/my-app:v1.0",
    "imageRegistryType": "personal"  # 个人版镜像仓库
}

# 创建沙箱时通过 metadata 传递 x-custom-config
sandbox = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(custom_image_config)
    }
)

print(f"沙箱ID: {sandbox.sandbox_id}")
print(f"沙箱信息: {sandbox.get_info()}")

## 执行命令


In [ ]:
# 创建带环境变量的沙箱
sandbox = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(env_config)
    }
)

print(f"沙箱信息: {sandbox.get_info()}")

# 运行命令
res = sandbox.commands.run(user="root", cmd="echo 'Hello World'")
print(res)

# 清理
sandbox.kill()

## 配置资源限制

可以通过 `resources` 字段配置沙箱的 CPU 和内存限制。

In [ ]:
# 定义资源限制配置
resource_config = {
    "resources": {
        "cpu": "2",      # 2核CPU
        "memory": "4Gi"  # 4GB内存
    }
}

# 创建带资源限制的沙箱
sandbox_with_resources = Sandbox.create(
    template="",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(resource_config)
    }
)

print(f"沙箱已创建，资源配置: CPU={resource_config['resources']['cpu']}, Memory={resource_config['resources']['memory']}")

# 清理
sandbox_with_resources.kill()

## 配置端口

可以通过 `ports` 字段配置容器暴露的端口。

In [ ]:
# 定义端口配置
port_config = {
    "ports": [
        {"name": "http", "port": 8080, "protocol": "TCP"},
        {"name": "metrics", "port": 9090, "protocol": "TCP"},
        {"name": "debug", "port": 5005, "protocol": "TCP"}
    ]
}

# 创建带端口配置的沙箱
sandbox_with_ports = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(port_config)
    }
)

# 获取各端口的访问地址
for port_info in port_config["ports"]:
    host = sandbox_with_ports.get_host(port_info["port"])
    print(f"{port_info['name']} 端口 ({port_info['port']}): {host}")

# 清理
sandbox_with_ports.kill()

## 配置健康检查

可以通过 `probe` 字段配置 HTTP 健康检查探针，用于监控沙箱内服务的健康状态。

In [ ]:
# 定义健康检查配置
probe_config = {
    "probe": {
        "httpGet": {
            "path": "/health",
            "port": 8080,
            "scheme": "HTTP"
        },
        "readyTimeoutMs": 30000,    # 就绪超时时间：30秒
        "probeTimeoutMs": 5000,     # 探针超时时间：5秒
        "probePeriodMs": 10000,     # 探针检查周期：10秒
        "successThreshold": 1,      # 成功阈值
        "failureThreshold": 3       # 失败阈值
    }
}

# 创建带健康检查的沙箱
sandbox_with_probe = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(probe_config)
    }
)

print("沙箱已创建，配置了健康检查:")
print(f"  - 检查路径: {probe_config['probe']['httpGet']['path']}")
print(f"  - 检查端口: {probe_config['probe']['httpGet']['port']}")
print(f"  - 就绪超时: {probe_config['probe']['readyTimeoutMs']}ms")
print(f"  - 失败阈值: {probe_config['probe']['failureThreshold']}")

# 清理
sandbox_with_probe.kill()

## 配置自定义启动命令

可以通过 `command` 和 `args` 字段覆盖镜像的默认 ENTRYPOINT 和 CMD。

In [ ]:
# 定义自定义启动命令配置
command_config = {
    "command": ["/bin/sh", "-c"],
    "args": ["echo 'Starting custom app...' && ./start.sh"]
}

# 创建带自定义启动命令的沙箱
sandbox_with_command = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(command_config)
    }
)

print("沙箱已创建，配置了自定义启动命令:")
print(f"  - Command: {command_config['command']}")
print(f"  - Args: {command_config['args']}")

# 清理
sandbox_with_command.kill()

## 综合配置示例

以下示例展示如何组合多种配置创建一个功能完整的自定义沙箱。

In [ ]:
# 定义综合配置
full_custom_config = {
    # 自定义镜像
    "image": "ccr.ccs.tencentyun.com/my-repo/my-app:v1.0",
    "imageRegistryType": "enterprise",
    
    # 启动命令
    "command": ["/bin/sh", "-c"],
    "args": ["./start.sh"],
    
    # 环境变量
    "env": [
        {"name": "NODE_ENV", "value": "production"},
        {"name": "LOG_LEVEL", "value": "info"},
        {"name": "MAX_WORKERS", "value": "4"}
    ],
    
    # 端口配置
    "ports": [
        {"name": "http", "port": 8080, "protocol": "TCP"},
        {"name": "metrics", "port": 9090, "protocol": "TCP"}
    ],
    
    # 资源限制
    "resources": {
        "cpu": "2",
        "memory": "4Gi"
    },
    
    # 健康检查
    "probe": {
        "httpGet": {
            "path": "/health",
            "port": 8080,
            "scheme": "HTTP"
        },
        "readyTimeoutMs": 30000,
        "probeTimeoutMs": 5000,
        "probePeriodMs": 10000,
        "successThreshold": 1,
        "failureThreshold": 3
    }
}

# 打印配置信息
print("完整自定义配置:")
print(json.dumps(full_custom_config, indent=2, ensure_ascii=False))

In [ ]:
# 使用综合配置创建沙箱
sandbox_full = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(full_custom_config)
    }
)

print(f"沙箱创建成功!")
print(f"沙箱ID: {sandbox_full.sandbox_id}")

# 获取沙箱信息
info = sandbox_full.get_info()
print(f"\n沙箱信息:")
print(f"  - 启动时间: {info.started_at}")
print(f"  - 结束时间: {info.end_at}")

## 查询沙箱获取配置信息

查询接口会在响应的 `metadata` 中返回 `x-custom-config`，并且会包含额外的 `imageDigest` 字段。

In [ ]:
# 获取沙箱列表并查看配置信息
paginator = Sandbox.list(limit=5)

print("沙箱列表及配置信息:")
for sandbox_info in paginator.next_items():
    print(f"\n沙箱ID: {sandbox_info.sandbox_id}")
    if hasattr(sandbox_info, 'metadata') and sandbox_info.metadata:
        if 'x-custom-config' in sandbox_info.metadata:
            config = json.loads(sandbox_info.metadata['x-custom-config'])
            print(f"  自定义配置:")
            if 'image' in config:
                print(f"    - 镜像: {config['image']}")
            if 'imageDigest' in config:
                print(f"    - 镜像摘要: {config['imageDigest']}")
            if 'resources' in config:
                print(f"    - 资源: CPU={config['resources'].get('cpu')}, Memory={config['resources'].get('memory')}")

## 错误处理

当 `x-custom-config` 格式错误时，API 会返回 HTTP 400 错误。

In [ ]:
# 示例：处理配置错误
try:
    # 错误的 JSON 格式
    invalid_sandbox = Sandbox.create(
        template="your-sandboxtool-name",
        timeout=3600,
        metadata={
            "x-custom-config": "invalid json format"
        }
    )
except Exception as e:
    print(f"创建失败，错误信息: {e}")
    print("\n提示: x-custom-config 必须是有效的 JSON 字符串")

In [ ]:
# 正确的做法：使用 json.dumps 确保格式正确
def create_custom_sandbox(config: dict, template: str = "your-sandboxtool-name", timeout: int = 3600):
    """
    创建自定义配置的沙箱
    
    Args:
        config: 自定义配置字典
        template: 沙箱模板
        timeout: 超时时间（秒）
    
    Returns:
        Sandbox 实例
    """
    try:
        # 使用 json.dumps 确保配置格式正确
        config_str = json.dumps(config)
        
        sandbox = Sandbox.create(
            template=template,
            timeout=timeout,
            metadata={
                "x-custom-config": config_str
            }
        )
        return sandbox
    except Exception as e:
        print(f"创建沙箱失败: {e}")
        return None

# 使用封装函数创建沙箱
my_config = {
    "env": [{"name": "MY_VAR", "value": "test"}],
    "resources": {"cpu": "1", "memory": "2Gi"}
}

sandbox = create_custom_sandbox(my_config)
if sandbox:
    print(f"沙箱创建成功: {sandbox.sandbox_id}")
    sandbox.kill()

## 总结

本教程介绍了如何使用 `x-custom-config` 扩展配置创建自定义沙箱，主要内容包括：

1. **自定义镜像**: 使用 `image` 和 `imageRegistryType` 指定自定义镜像
2. **环境变量**: 使用 `env` 配置环境变量列表
3. **资源限制**: 使用 `resources` 配置 CPU 和内存限制
4. **端口配置**: 使用 `ports` 配置容器端口
5. **健康检查**: 使用 `probe` 配置 HTTP 健康检查探针
6. **启动命令**: 使用 `command` 和 `args` 自定义启动命令

### 最佳实践

- 始终使用 `json.dumps()` 将配置字典转换为 JSON 字符串
- 根据实际需求合理配置资源限制